In [3]:
import numpy as np
import pandas as pd
import json
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

### Demographic analysis for the study1 

*NOTE*: Please run the code at the bottom first 

### Age

In [68]:
age_ranges = [i for i in range(0, 101, 10)]
all_demo_df.groupby(pd.cut(all_demo_df.age, age_ranges, right=False)).count()["id"]

age
[0, 10)       0
[10, 20)      1
[20, 30)     22
[30, 40)     15
[40, 50)     15
[50, 60)      5
[60, 70)      3
[70, 80)      0
[80, 90)      0
[90, 100)     0
Name: id, dtype: int64

In [69]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,6))

count, division = np.histogram(all_demo_df.age, bins = age_ranges)
all_demo_df.age.hist(bins=division)

plt.ylabel("count")
plt.xlabel("age")

### vision

In [70]:
all_demo_df["vision"].value_counts()

yes    54
no      7
Name: vision, dtype: int64

In [71]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,6))
sns.countplot(x="vision", data=all_demo_df)

### education

In [72]:
all_demo_df["education"].value_counts()

collegeGraduate    21
someCollege        19
highSchool         12
graduateStudies     9
Name: education, dtype: int64

In [73]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,6))
ax = sns.countplot(x="education", data=all_demo_df)

### gender

In [74]:
all_demo_df["gender"].value_counts()

Male       31
Female     28
noReply     2
Name: gender, dtype: int64

In [30]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,6))
ax = sns.countplot(x="gender", data=all_demo_df)

### occupation

In [31]:
all_demo_df["occupation"].value_counts()

Engineer                      3
manager                       2
student                       2
automotive mechanic           2
Manager                       2
mturkey                       1
Manfacturing                  1
Teacher                       1
RN                            1
Home maker                    1
Legal Receptionist            1
health                        1
Data Processor                1
retail                        1
SAHM                          1
house keeper                  1
Designer                      1
retired                       1
Self Employed                 1
Graduate student              1
Civil Engineering             1
project manager               1
Amazon Mechanical Turks       1
Self-empolyed                 1
sales                         1
Doctor                        1
Case Worker                   1
merchandiser                  1
PC Support                    1
data entry clerk              1
Student                       1
machinis

In [32]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,6))
ax = sns.countplot(x="occupation", data=all_demo_df)
l = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

### Run all code below first

In [16]:
def read_data(datasets, user_data_path, data_paths):
    """
    Read the data from the file 
    (all of this functions are from Michael's code)
    """
    
    ## Demographics
    demographics_header = ["id", "age", "vision", "vcd", "comp_time", "education", "disply_type", "gender", "occupation", "monitor_size", "comments"]
    demographics_dfs_raw = []
    for dataset_id in datasets:
        df_raw = pd.read_csv(data_paths["demographics"][dataset_id], header=None)

        # Set header
        df_raw.columns = demographics_header

        # Join A1
        df_processed = df_raw.join(pd.DataFrame({'Dataset': [dataset_id for _ in range(len(df_raw))]}))

        if demographics_dfs_raw != []:
            previous_length = len(demographics_dfs_raw[-1])
            current_length = len(df_processed)
            new_upper_bound = previous_length + current_length
            df_processed.index = [i for i in range(previous_length, new_upper_bound)]        

        # Store
        demographics_dfs_raw.append(df_processed)
    
    
    ## Join Dataframes for subsets
    demographics_dfs = pd.concat(demographics_dfs_raw, axis=0)
    
    response_data_header = ["User ID", "Question Number", "Question", "Breed Key", "Image Name", "User Response", "Correct Response", "Response Time"]
    response_data_dfs_raw = []
    for dataset_id in datasets:
        df_raw = pd.read_csv(data_paths["response_data"][dataset_id], header=None)

        # Set header
        df_raw.columns = response_data_header

        # Join A1
        df_processed = df_raw.join(pd.DataFrame({'Dataset': [dataset_id for _ in range(len(df_raw))]}))

        if response_data_dfs_raw != []:
            previous_length = len(response_data_dfs_raw[-1])
            current_length = len(df_processed)
            new_upper_bound = previous_length + current_length
            df_processed.index = [i for i in range(previous_length, new_upper_bound)]        

        # Store
        response_data_dfs_raw.append(df_processed)

    response_data_dfs = pd.concat(response_data_dfs_raw, axis=0)
    
    
    user_IDs = set(response_data_dfs["User ID"])

    invalid_users = {}
    valid_user_ids = []
    for user_id in user_IDs:

        user_is_valid = True
        fail_reasons = []
        user_df = response_data_dfs[response_data_dfs["User ID"] == user_id]

        # CHECK NUMBER OF USER ENTRIES! Ensure 103
        # ========================================================
        num_user_entries = len(user_df)

        if num_user_entries < 103:
            fail_reason = "User {} does not have 103 entries - they have {}.".format(user_id, num_user_entries)
            fail_reasons.append(fail_reason)
            user_is_valid = False

        elif num_user_entries == 206:
            fail_reason = "User {} has 206 entries! Readjusting dataframe.".format(user_id)

            unique_sets = set(user_df["Dataset"])
            if unique_sets == 2:
                keep_set = datasets[np.random.randint(2)]
                user_df = user_df[user_df["Dataset"] == keep_set]
            else:
                user_df = user_df[:103]
        # ========================================================

        # CHECK THAT USERS DID NOT ENTER ONLY 1 ANSWER: e.g., all B's
        # ========================================================
        user_responses = user_df["User Response"]
        unique_responses = set(user_responses)

        if len(unique_responses) == 1:
            fail_reason = "User {} only entered {}'s".format(user_id, list(unique_responses)[0])
            fail_reasons.append(fail_reason)
            user_is_valid = False
        # ========================================================

        # CHECK THAT USERS PASSED ATTENTION CHECKS
        # ========================================================
        attention_check_threshold = 2
        attention_check_correct_responses = user_df[user_df["Image Name"].str.contains('attention_check')]["Correct Response"]
        attention_checks_passed = len(np.where(attention_check_correct_responses == True)[0])
        if attention_checks_passed < attention_check_threshold:
            fail_reason = "User {} only passed {}/4 attention checks!".format(user_id, attention_checks_passed)
            fail_reasons.append(fail_reason)
            user_is_valid = False
        # ========================================================

        if not user_is_valid:
            invalid_users[user_id] = {"reasons" : fail_reasons}
        else:
            valid_user_ids.append(user_id)

    invalid_user_ids = invalid_users.keys()

    num_total_users = len(user_IDs)
    print("Total users: {}".format(num_total_users))

    print("\n")

    print("Invalid Users")
    print("-------------")
    for user_i, (invalid_user_id, val) in enumerate(invalid_users.items(), 1):
        print("\t{}. User {} -- Reasons: {}".format(user_i, invalid_user_id, val["reasons"][0]))

    print("\n")

    print("Valid Users")
    print("-----------")
    for valid_i, valid_user_id in enumerate(valid_user_ids):
        print("\t{:2.0f}. User {}".format(valid_i, valid_user_id))

    
    valid_users = np.array(valid_user_ids).astype(np.int64)
    valid_users
    
    demo_df = demographics_dfs[demographics_dfs["id"].isin(valid_users)]
    demo_df
    
    return demo_df
    

In [17]:
def make_path(KEY1, KEY2): 
    user_data_path = "data/study/user_data.csv"
    data_paths = {
        "response_data" : {
            KEY1 : "data/study/response_data_{}.csv".format(KEY1),
            KEY2 : "data/study/response_data_{}.csv".format(KEY2)
        },
        "demographics" : {
            KEY1 : "data/study/demographics_data_{}.csv".format(KEY1),
            KEY2 : "data/study/demographics_data_{}.csv".format(KEY2)
        }
    }
    return user_data_path, data_paths

### Read each data set and combine them

In [63]:
KEY1 = "A1"
KEY2 = "A2"
datasets = [KEY1, KEY2]

user_data_path, data_paths = make_path(KEY1, KEY2)
demo_df_setA = read_data(datasets, user_data_path, data_paths)

Total users: 31


Invalid Users
-------------
	1. User 66 -- Reasons: User 66 only passed 0/4 attention checks!
	2. User 3 -- Reasons: User 3 only passed 0/4 attention checks!
	3. User 37 -- Reasons: User 37 only passed 0/4 attention checks!
	4. User 7 -- Reasons: User 7 only passed 0/4 attention checks!
	5. User 8 -- Reasons: User 8 only passed 0/4 attention checks!
	6. User 9 -- Reasons: User 9 only passed 1/4 attention checks!
	7. User 48 -- Reasons: User 48 only passed 0/4 attention checks!
	8. User 17 -- Reasons: User 17 only passed 0/4 attention checks!
	9. User 52 -- Reasons: User 52 only passed 0/4 attention checks!
	10. User 53 -- Reasons: User 53 only passed 0/4 attention checks!
	11. User 57 -- Reasons: User 57 only passed 0/4 attention checks!


Valid Users
-----------
	 0. User 18
	 1. User 21
	 2. User 22
	 3. User 23
	 4. User 26
	 5. User 30
	 6. User 32
	 7. User 34
	 8. User 40
	 9. User 43
	10. User 44
	11. User 49
	12. User 50
	13. User 55
	14. User 56
	15. User 60


In [64]:
KEY1 = "B1"
KEY2 = "B2"
datasets = [KEY1, KEY2]

user_data_path, data_paths = make_path(KEY1, KEY2)
demo_df_setB = read_data(datasets, user_data_path, data_paths)

Total users: 33


Invalid Users
-------------
	1. User 115 -- Reasons: User 115 only passed 1/4 attention checks!
	2. User 70 -- Reasons: User 70 only passed 1/4 attention checks!
	3. User 137 -- Reasons: User 137 only passed 0/4 attention checks!
	4. User 113 -- Reasons: User 113 only passed 0/4 attention checks!
	5. User 146 -- Reasons: User 146 only passed 0/4 attention checks!
	6. User 83 -- Reasons: User 83 only passed 1/4 attention checks!
	7. User 117 -- Reasons: User 117 only passed 0/4 attention checks!
	8. User 118 -- Reasons: User 118 only passed 0/4 attention checks!
	9. User 151 -- Reasons: User 151 only passed 1/4 attention checks!
	10. User 89 -- Reasons: User 89 only passed 1/4 attention checks!
	11. User 158 -- Reasons: User 158 only passed 1/4 attention checks!
	12. User 159 -- Reasons: User 159 only passed 0/4 attention checks!


Valid Users
-----------
	 0. User 134
	 1. User 136
	 2. User 143
	 3. User 147
	 4. User 148
	 5. User 155
	 6. User 160
	 7. User 165
	 8

In [89]:
KEY1 = "C1"
KEY2 = "C2"
datasets = [KEY1, KEY2]

user_data_path, data_paths = make_path(KEY1, KEY2)
demo_df_setC = read_data(datasets, user_data_path, data_paths)

Total users: 35


Invalid Users
-------------
	1. User 192 -- Reasons: User 192 only passed 0/4 attention checks!
	2. User 161 -- Reasons: User 161 only passed 0/4 attention checks!
	3. User 166 -- Reasons: User 166 only passed 0/4 attention checks!
	4. User 172 -- Reasons: User 172 only passed 1/4 attention checks!
	5. User 170 -- Reasons: User 170 only passed 1/4 attention checks!
	6. User 139 -- Reasons: User 139 only passed 0/4 attention checks!
	7. User 140 -- Reasons: User 140 only passed 0/4 attention checks!
	8. User 174 -- Reasons: User 174 only passed 0/4 attention checks!
	9. User 143 -- Reasons: User 143 only passed 1/4 attention checks!
	10. User 144 -- Reasons: User 144 only passed 0/4 attention checks!
	11. User 177 -- Reasons: User 177 only passed 1/4 attention checks!
	12. User 83 -- Reasons: User 83 only passed 0/4 attention checks!
	13. User 180 -- Reasons: User 180 only passed 1/4 attention checks!
	14. User 85 -- Reasons: User 85 only passed 0/4 attention checks!
	

In [90]:
## Combine the data 
all_demo_df = pd.concat([demo_df_setA, demo_df_setB, demo_df_setC], axis=0)

In [91]:
all_demo_df.head()

id  age vision vcd  comp_time        education disply_type  gender  \
0  22   46    yes  no          6  collegeGraduate      laptop    Male   
1  23   43    yes  no         10  collegeGraduate     desktop  Female   
3  30   25    yes  no          7  graduateStudies     desktop    Male   
5  32   35    yes  no          6       highSchool      laptop    Male   
6  32   35    yes  no          6       highSchool      laptop    Male   

                occupation  monitor_size comments Dataset  
0                    Sales            13      NaN      A1  
1  Client Services Manager            24      NaN      A1  
3         Graduate student            20      NaN      A1  
5      automotive mechanic            15      NaN      A1  
6      automotive mechanic            15      NaN      A1

In [84]:
A_age = list(all_demo_df["age"])

In [88]:
B_age = list(all_demo_df["age"])

In [92]:
C_age = list(all_demo_df["age"])

In [93]:
ages = A_age + B_age + C_age

In [94]:
np.mean(ages)

37.72826086956522

In [95]:
np.std(ages)

12.554319593418281